# Try SSLEM
Running the SSLEM example script from pyechomask with my own data.

In [31]:
## import packages
import glob
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import echopype as ep
import xarray as xr
from pyechoplot.plotting import plot_Sv, plot_mask, save_png_plot, setup_ek500_cmap

## import pyechomask modules
from pyechomask.masks import binary_seabed, binary_signal
from pyechomask.manipulate import  signal_row_filter, signal_column_filter, flag,\
        vertical_merge,feature_median, fill_feature_gaps, break_mask, remove_features

%matplotlib inline

In [2]:
## Bug fix for missing colourmap
def plot_mask_fix(mask):
    '''
    plot mask (any - binary/flag/continuous)
    '''
    ## shape
    row,col = mask.shape
    
    ## plot
    #f, (ax1) = plt.subplots(1, figsize = (20,10))
    p1       = plt.imshow(mask, cmap = plt.cm.nipy_spectral,\
            interpolation='nearest',aspect='auto')
    plt.colorbar(p1,pad = 0)
    plt.xlabel('columns',fontsize = 18)
    plt.ylabel('rows',fontsize = 18)

In [3]:
def plot_SSLEM(Sv,mask = None, sample_int=0.5):
    '''
    :param Sv: gridded Sv values (dB re 1m^-1)
    :type  Sv: 2D numpy.array
    
    :param mask: binary mask (0 - noise; 1 - signal)
    :type  mask: 2D numpy.array
    
    return:
    
    desc: Plot Sv grid, with/without binary mask
    
    defined by RP
    
    status: dev
    
    '''
    ## get echoview colormap
    setup_ek500_cmap()
    ek500_cmap = mpl.cm.get_cmap('ek500')
    ek500_norm = mpl.colors.BoundaryNorm(np.linspace(-89,-34,12), 12, clip=False)
    
    ## add mask
    if mask is not None:
        Sv = np.ma.masked_where(mask == 0,Sv)
    ## shape
    row,col = Sv.shape
    
    ## plot
    #f, (ax1) = plt.subplots(1, figsize = (20,10))
    p1       = plt.imshow(Sv, cmap = ek500_cmap,norm = ek500_norm,\
            interpolation='nearest',aspect='auto',extent=[0,col,row*sample_int,0])
    
    plt.colorbar(p1,pad = 0)
    plt.xlabel('Ping Number',fontsize = 18)
    plt.ylabel('Depth',fontsize = 18)

## SSLEM Parameters

In [4]:
## Sv observation parameters
noise_level   = -999   # background noise level (dB re 1m^-1)
sample_int    = 0.5    # sample interval (m)
pl            = 1.024 # pulse length (ms)
fq            = 333   # Frequency

## SSLEM optimization parameters (recomended setting for regional analysis)
min_sep       = 20     # minimum SSL seperation (m):min = pulse length
max_thickness = 70    # Maximun SSL thickness  (m)
min_size      = 10     # Minimun thickness of SSL (pings)
min_thickness = 10     # Minimun thickness of SSL (m)

## thresholds 
minSv = -85
maxSv = -40

In [47]:
def SSLEM_nc(ds, 
                 sample_int=1, 
                 pl=1.024, 
                 fq=333, 
                 min_sep=15, 
                 min_size=15, 
                 min_thickness=5, 
                 max_thickness=70, 
                 minSv=-90, 
                 maxSv=-50):
    '''
    Apply SSL evaluation model to an Echoview cell export .xlsx file.
    
    ## Sv observation parameters
    noise_level   = -999   # background noise level (dB re 1m^-1)
    sample_int    = 0.5    # sample interval (m)
    pl            = 1.024 # pulse length (ms)
    fq            = 333   # Frequency

    ## SSLEM optimization parameters (recomended setting for regional analysis)
    min_sep       = 10     # minimum SSL seperation (m):min = pulse length
    max_thickness = 70    # Maximun SSL thickness  (m)
    min_size      = 25    # Minimum duration of SSL (pings)
    min_thickness = 10     # Minimun thickness of SSL (m)
    
    ## thresholds 
    minSv_boat = -90
    maxSv_boat = -40 
    '''
    # Extract Sv numpy array from data
    Sv = np.array(ds.Sv.isel(frequency=0)).T

    ## calculate number of rows/samples from meters
    min_thickness_rows = int(min_thickness/sample_int)
    min_sep_rows       = int(min_sep/sample_int)

    
    #---------------------------------
    ## SSLEM
    ## remove pulse (by pulse does he mean surface? They removed first 100m...)
    Sv[0:2,:] = noise_level  

    ## remove weak signal
    Sv[Sv < minSv] = noise_level


    ## get seabed mask, window_size refers to a rolling median filter
    ## seabed mask in development - check
    #seabed_mask, seabed_idx = binary_seabed(Sv, buffer = min_sep_rows, window_size = 10) # No seabed here.

    
    
    #-------------------------------------
    ## Identify signal pixels
    ## this step is the bottle neck
    signal_mask = binary_signal(Sv,pl,sample_int,min_sep,max_thickness,max_steps = 10)

    ## row smooth signal mask:
    ## where majority (threshold = 0.5) pixels are signal make all signal, otherwise label all noise
    signal = signal_row_filter(signal_mask,min_size,threshold = 0.5) 

    ## remove SSLs that are too thin
    signal = signal_column_filter(signal,min_thickness_rows,threshold = 1)


    ## label SSls (flag) and remove small SSLs (size < min_size*min_thickness_rows)
    signal = flag(signal,min_size*min_thickness_rows)

    ## merge SSLs that are closer together than the min seperation
    signal[signal > 0] = 1
    signal             = vertical_merge(signal,min_sep_rows)

    ## fill internal gaps - max size = min_size*min_thickness_rows
    signal = fill_feature_gaps(signal,min_size*min_thickness_rows)

    ## break merged SSLs into individual SSLs
    signal = break_mask(signal)

    ## remove small SSLs
    signal = remove_features(signal.astype(int),min_size*min_thickness_rows)

    ## get median values of each SSL
    Sv_median = feature_median(Sv,signal)

    
    
    #--------------------------------------
    ## plot results
    plt.figure(figsize=(15,10))
    plt.subplot(311)
    plot_Sv(Sv)
    plt.title('333kHz echogram SSLEM: echogram, SSL mask and SSL median')
    plt.subplot(312)
    plot_mask_fix(signal) 
    plt.subplot(313)
    plot_SSLEM(Sv_median) 
    plt.show()
    return 

# Load nc file using echopype

In [48]:
fname = 'D:/GLIDER2018/Echosounder/Echosounder/nc_files/GLIDER_SB_2018_v5-Phase0-D20180625-T170658-1.nc'
ed = ep.open_converted(fname)
ds = ep.calibrate.compute_Sv(ed, waveform_mode='BB', encode_mode='complex')
#ds.coords['range_bin']=np.array(ds.range[0,0,:])

<string>:6: DeprecationWarning: `np.alen` is deprecated, use `len` instead


In [ ]:
SSLEM_nc(Sv, fq=333, minSv=-80)#, sample_int=sample_int_boat, pl=pl_boat, fq=fq_boat, min_sep=min_sep_boat, min_size=min_size_boat, min_thicknes=min_thickness_boat, max_thickness=max_thickness_boat, minSv=minSv_boat, maxSv=maxSv_boat)

C:\Users\mbd\anaconda3\lib\site-packages\ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in less


 Volume in drive C is OS
 Volume Serial Number is C619-249F

 Directory of C:\Users\mbd\phd

31.03.2021  14:13    <DIR>          .
31.03.2021  14:13    <DIR>          ..
15.02.2021  11:38               157 .gitignore
13.01.2021  15:25    <DIR>          .ipynb_checkpoints
03.03.2021  12:00    <DIR>          __pycache__
04.11.2020  11:59             1 006 acousticTS.Rmd
06.10.2020  13:29    <DIR>          ANMPAanalysis
26.03.2021  15:14    <DIR>          BIO8514
08.12.2020  11:24           487 836 BlindZone_NASC.ipynb
04.12.2020  12:39           266 555 continuum_28AM.png
07.12.2020  11:21       223 016 320 CRIMAC_EK80Test_GOSars.raw
13.01.2021  15:18           154 813 CruiseData_Tromsøflaket2018.ipynb
28.01.2021  03:49             1 160 debug.log
08.12.2020  09:56             6 295 explore_pyEcholab.ipynb
13.01.2021  14:53           111 094 figure0.png
13.01.2021  14:53           111 393 figure1.png
13.01.2021  14:55           113 128 figure10.png
13.01.2021  15:09           119 739 fig

13.01.2021  15:08           119 388 figure90.png
13.01.2021  15:08           119 537 figure91.png
13.01.2021  15:08           119 503 figure92.png
13.01.2021  15:08           119 483 figure93.png
13.01.2021  15:08           119 533 figure94.png
13.01.2021  15:08           119 486 figure95.png
13.01.2021  15:09           119 695 figure96.png
13.01.2021  15:09           119 631 figure97.png
13.01.2021  15:09           119 719 figure98.png
13.01.2021  15:09           119 864 figure99.png
23.11.2020  11:45    <DIR>          figures
30.07.2020  10:53             2 203 Gray nodata.evc
18.12.2020  15:58           433 257 Multi_species.png
18.12.2020  15:22             3 244 Multinet_species.png
24.06.2020  13:06    <DIR>          phd-env
13.01.2021  15:18            82 270 plot_contour_Tromsøflaket.ipynb
15.02.2021  14:37           102 474 plot_Tromsoflaket.ipynb
15.02.2021  14:43    <DIR>          Proposal
07.12.2020  10:24    <DIR>          pyEcholab
07.04.2020  10:20                32 READ